In [1]:
import os, sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
# --- 
from techminer2 import *

directory = "regtech"

import_scopus_files(directory)
# import_references()
# annual_scientific_production()
# main_information(directory)

Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'karateclub', 'graph_tool', 'wurlitzer', 'infomap'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'ASLPAw', 'karateclub'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'leidenalg', 'infomap', 'wurlitzer'}
--INFO-- Concatenating raw files in regtech/raw/cited_by/
--INFO-- Concatenating raw files in regtech/raw/references/
--INFO-- Concatenating raw files in regtech/raw/documents/
--INFO-- Applying scopus tags to database files
--INFO-- Formating column names in database files
--INFO-- Dropping NA columns in database files
--INFO-- Removing accents in database files
--INFO-- Creating `filter.yaml` file
--INFO-- Processing `abstract` column
--INFO-- Processing `authors_id` column
--INFO-- Processing `title` column
--INFO-- Processing `document_type` column
--INFO-- Processing `doi` column

In [3]:
import pandas as pd

data = pd.read_csv(f"{directory}/processed/_documents.csv")
data.title_words

0     bank; digital transformation; regtech; regulat...
1     application; blockchain; india; manage; report...
2     bahrain; laundering; money; money laundering; ...
3     ai; artificial intelligence; financial; financ...
4     bibliometric; environment; financial; financia...
                            ...                        
89                            robo-advisors; taxonomies
90                               blockchain; compliance
91          data; ecosystem; fintech; fintech ecosystem
92                 compliance; legal; legal rules; rule
93     finance; finance industries; industrial; regtech
Name: title_words, Length: 94, dtype: object

In [4]:
import pandas as pd

data = pd.read_csv(f"{directory}/processed/_documents.csv")
for t in data.raw_title_words.head():
    print(t)

bank; digital transformation; regtech; regulatory technology; technology; transformation
application; blockchain; india; management; reporting framework
bahrain; laundering; money; money laundering; prevention; regtech; regulatory technology; technology
ai; artificial intelligence; financial; financial technology; fintech; intelligence; pandemic; technology
bibliometric; environment; financial; financial technology; fintech; society; technology; trends


In [3]:
import pandas as pd

data = pd.read_csv(f"{directory}/processed/_documents.csv")
for t in data.title.head():
    print(t)


Integrating Regulatory Technology (RegTech) into the digital transformation of a bank Treasury
Corporate bankingrisk management, regulatory and reporting framework in India: a Blockchain application-based approach
Regulatory technology (regtech) and money laundering prevention: exploratory study from bahrain
Artificial Intelligence and Financial Technology FinTech: How AI Is Being Used Under the Pandemic in 2020
Trends and directions of financial technology (Fintech) in society and environment: A bibliometric study
